1 subject. Feature extraction. Classification joining all frequency bands.

In [448]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

#def select_file(file):
    #Lectura mediante pandas
f = "data_preprocessed_python\\s32.dat"
subject_id = 32
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

In [449]:
    # Frecuencia de muestreo
    fs = 128
    # Window
    window = "hann"
    # Length of each segment
    # nperseg = 256 por defecto
    # noverlap
    # Por defecto a None, if None : noverlap = nperseg / 2
    # [...]

    # Definicion de bandas
    eeg_bands = {'Delta': (1, 4),
                'Theta': (4, 8),
                'Alpha': (8, 14),
                'Beta': (14, 30),
                'Gamma': (30, 50)}

In [450]:
# FOURIER
import scipy
from scipy import signal

# SE COGE EL PRIMER VÍDEOS, Df_VIDEOS[0]
def do_stft(video, channel):
    # Array 
    x = df_videos[video][channel]

    # Array of the sample frequency, Array of the segment times, STFT of x
    f, t , Zxx = scipy.signal.stft(x, fs, window)

    #print(np.abs(Zxx))
    '''
    plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    '''
    return Zxx


In [451]:
#SEPARACION EN LAS DISTINTAS BANDAS DE FRECUENCIA

def make_bands(Zxx):
    #Obtener valores reales de STFT, solo positivos.
    values = np.absolute(Zxx)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
    #print(fft_freq)

    # Obtención de cada una de las bandas de frecuencia
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = values[freq_ix]

    #print(eeg_band_fft)
    '''
    eeg_band_fft_means = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft_means[band] = np.mean(values[freq_ix])

    #print(eeg_band_fft_means)

    #Graficación de las medias de los datos
    df = pd.DataFrame(columns=['band', 'val'])
    df['band'] = eeg_bands.keys()
    df['val'] = [eeg_band_fft_means[band] for band in eeg_bands]
    ax = df.plot.bar(x='band', y='val', legend=False)
    ax.set_xlabel("EEG band")
    ax.set_ylabel("Mean band Amplitude")
    '''
    #print(df)

    #print("\n\n")

    '''
    print("Shape de Zxx: ", Zxx.shape)
    print("Shape de Delta: ", eeg_band_fft['Delta'].shape)
    print("Shape de Theta: ", eeg_band_fft['Theta'].shape)
    print("Shape de Alpha: ", eeg_band_fft['Alpha'].shape)
    print("Shape de Beta: ", eeg_band_fft['Beta'].shape)
    print("Shape de Gamma: ", eeg_band_fft['Gamma'].shape)
    '''
    return eeg_band_fft


In [452]:
# Ahora hay que hacer las medias sobre cada banda de frecuencia. La media entre cada una de las posiciones de los bloques. Hacer la media entre todas las posiciones 0, 1, 2... 63 de cada uno de los bloques
import numpy as np

def channel_freq(eeg_band_fft):
        
        freq = np.zeros((5, 64))        # 5 bandas, 64 posiciones de los datos

        # Se va a recorrer {Delta, Theta, Alpha, Beta, Gamma}
        for band in eeg_bands:

                # Para cada una de las 64 posiciones del array
                for j in range (0, 64):
                        
                        val = []
                        # Se va a recorrer cada uno de los arrays que hay en cada una de las bandas
                        for i in range (0, len(eeg_band_fft[band])):
                                val.append(eeg_band_fft[band][i][j])
                                # Ejemplo : eeg_band_fft['Delta'][0][0*0 + 0]
                        if (band == "Delta"):
                                freq[0][j] = np.mean(val)
                        elif (band == "Theta"):
                                freq[1][j] = np.mean(val)
                        elif (band == "Alpha"):
                                freq[2][j] = np.mean(val)
                        elif (band == "Beta"):
                                freq[3][j] = np.mean(val)
                        elif (band == "Gamma"):
                                freq[4][j] = np.mean(val)
        return(freq)     # En freq tenemos las frecuencias medias de cada una de las bandas

# Recordamos : Estamos tratando los datos relativos al video 0, un canal concreto de un sujeto concreto

In [453]:
#df_file_videos = select_file('d\\s01.dat')
# Obtener la división por bandas de frecuencia de cada uno de los canales del video 0
for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])
        
#        print(freq[0])
 #       print("\n\n")
    # En freq tenemos la media de todas las bandas de frecuencia de cada uno de los canales del vídeo 0

In [454]:
# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

aux_delta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.003053,0.359062,0.012175,0.646743,0.056972,0.337213,0.031191,0.115313,0.055065,0.134096
1,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.003053,0.359062,0.012175,0.646743,0.056972,0.337213,0.031191,0.115313,0.055065,0.134096
2,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.003053,0.359062,0.012175,0.646743,0.056972,0.337213,0.031191,0.115313,0.055065,0.134096
3,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.003053,0.359062,0.012175,0.646743,0.056972,0.337213,0.031191,0.115313,0.055065,0.134096
4,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.003053,0.359062,0.012175,0.646743,0.056972,0.337213,0.031191,0.115313,0.055065,0.134096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,0.173321,0.091425,0.057593,0.169698,0.121709,0.140030,0.089309,0.139273,0.103451,0.205700
2556,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,0.173321,0.091425,0.057593,0.169698,0.121709,0.140030,0.089309,0.139273,0.103451,0.205700
2557,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,0.173321,0.091425,0.057593,0.169698,0.121709,0.140030,0.089309,0.139273,0.103451,0.205700
2558,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,0.173321,0.091425,0.057593,0.169698,0.121709,0.140030,0.089309,0.139273,0.103451,0.205700


In [455]:
# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

aux_theta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.392106,0.199871,0.159931,1.293292,0.201255,0.245633,0.120701,0.847831,0.300534,0.701090,...,0.007365,0.442002,0.015736,0.814071,0.056054,0.406192,0.028243,0.110069,0.050692,0.144697
1,0.392106,0.199871,0.159931,1.293292,0.201255,0.245633,0.120701,0.847831,0.300534,0.701090,...,0.007365,0.442002,0.015736,0.814071,0.056054,0.406192,0.028243,0.110069,0.050692,0.144697
2,0.392106,0.199871,0.159931,1.293292,0.201255,0.245633,0.120701,0.847831,0.300534,0.701090,...,0.007365,0.442002,0.015736,0.814071,0.056054,0.406192,0.028243,0.110069,0.050692,0.144697
3,0.392106,0.199871,0.159931,1.293292,0.201255,0.245633,0.120701,0.847831,0.300534,0.701090,...,0.007365,0.442002,0.015736,0.814071,0.056054,0.406192,0.028243,0.110069,0.050692,0.144697
4,0.392106,0.199871,0.159931,1.293292,0.201255,0.245633,0.120701,0.847831,0.300534,0.701090,...,0.007365,0.442002,0.015736,0.814071,0.056054,0.406192,0.028243,0.110069,0.050692,0.144697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.091273,0.058506,0.286167,0.292735,0.193007,0.110957,0.055552,0.303595,0.122076,0.194943,...,0.176813,0.107332,0.066196,0.162048,0.127523,0.194332,0.090885,0.139967,0.104995,0.227190
2556,0.091273,0.058506,0.286167,0.292735,0.193007,0.110957,0.055552,0.303595,0.122076,0.194943,...,0.176813,0.107332,0.066196,0.162048,0.127523,0.194332,0.090885,0.139967,0.104995,0.227190
2557,0.091273,0.058506,0.286167,0.292735,0.193007,0.110957,0.055552,0.303595,0.122076,0.194943,...,0.176813,0.107332,0.066196,0.162048,0.127523,0.194332,0.090885,0.139967,0.104995,0.227190
2558,0.091273,0.058506,0.286167,0.292735,0.193007,0.110957,0.055552,0.303595,0.122076,0.194943,...,0.176813,0.107332,0.066196,0.162048,0.127523,0.194332,0.090885,0.139967,0.104995,0.227190


In [456]:
#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

aux_alpha

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.455806,0.223610,0.173599,1.475523,0.228190,0.286358,0.136515,0.976923,0.343249,0.794884,...,0.022726,0.508893,0.020280,0.935351,0.057744,0.458464,0.029319,0.109012,0.047509,0.145896
1,0.455806,0.223610,0.173599,1.475523,0.228190,0.286358,0.136515,0.976923,0.343249,0.794884,...,0.022726,0.508893,0.020280,0.935351,0.057744,0.458464,0.029319,0.109012,0.047509,0.145896
2,0.455806,0.223610,0.173599,1.475523,0.228190,0.286358,0.136515,0.976923,0.343249,0.794884,...,0.022726,0.508893,0.020280,0.935351,0.057744,0.458464,0.029319,0.109012,0.047509,0.145896
3,0.455806,0.223610,0.173599,1.475523,0.228190,0.286358,0.136515,0.976923,0.343249,0.794884,...,0.022726,0.508893,0.020280,0.935351,0.057744,0.458464,0.029319,0.109012,0.047509,0.145896
4,0.455806,0.223610,0.173599,1.475523,0.228190,0.286358,0.136515,0.976923,0.343249,0.794884,...,0.022726,0.508893,0.020280,0.935351,0.057744,0.458464,0.029319,0.109012,0.047509,0.145896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.110731,0.069076,0.308047,0.263358,0.204854,0.115875,0.055528,0.310905,0.130438,0.195797,...,0.197670,0.106538,0.081266,0.139539,0.138271,0.207269,0.099220,0.151011,0.113075,0.249053
2556,0.110731,0.069076,0.308047,0.263358,0.204854,0.115875,0.055528,0.310905,0.130438,0.195797,...,0.197670,0.106538,0.081266,0.139539,0.138271,0.207269,0.099220,0.151011,0.113075,0.249053
2557,0.110731,0.069076,0.308047,0.263358,0.204854,0.115875,0.055528,0.310905,0.130438,0.195797,...,0.197670,0.106538,0.081266,0.139539,0.138271,0.207269,0.099220,0.151011,0.113075,0.249053
2558,0.110731,0.069076,0.308047,0.263358,0.204854,0.115875,0.055528,0.310905,0.130438,0.195797,...,0.197670,0.106538,0.081266,0.139539,0.138271,0.207269,0.099220,0.151011,0.113075,0.249053


In [457]:
# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

aux_beta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,1.116295,0.396166,0.265820,3.36259,0.515325,0.764339,0.307574,2.372068,0.820625,1.772782,...,0.309920,1.256967,0.194967,2.259644,0.128323,1.014706,0.138964,0.207237,0.243517,0.389165
1,1.116295,0.396166,0.265820,3.36259,0.515325,0.764339,0.307574,2.372068,0.820625,1.772782,...,0.309920,1.256967,0.194967,2.259644,0.128323,1.014706,0.138964,0.207237,0.243517,0.389165
2,1.116295,0.396166,0.265820,3.36259,0.515325,0.764339,0.307574,2.372068,0.820625,1.772782,...,0.309920,1.256967,0.194967,2.259644,0.128323,1.014706,0.138964,0.207237,0.243517,0.389165
3,1.116295,0.396166,0.265820,3.36259,0.515325,0.764339,0.307574,2.372068,0.820625,1.772782,...,0.309920,1.256967,0.194967,2.259644,0.128323,1.014706,0.138964,0.207237,0.243517,0.389165
4,1.116295,0.396166,0.265820,3.36259,0.515325,0.764339,0.307574,2.372068,0.820625,1.772782,...,0.309920,1.256967,0.194967,2.259644,0.128323,1.014706,0.138964,0.207237,0.243517,0.389165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.653332,0.387943,0.793178,1.30196,0.716276,0.394747,0.118335,1.410465,0.451186,0.969091,...,0.348772,1.011982,0.191880,1.230596,0.247684,0.946895,0.212751,0.228151,0.217311,0.757526
2556,0.653332,0.387943,0.793178,1.30196,0.716276,0.394747,0.118335,1.410465,0.451186,0.969091,...,0.348772,1.011982,0.191880,1.230596,0.247684,0.946895,0.212751,0.228151,0.217311,0.757526
2557,0.653332,0.387943,0.793178,1.30196,0.716276,0.394747,0.118335,1.410465,0.451186,0.969091,...,0.348772,1.011982,0.191880,1.230596,0.247684,0.946895,0.212751,0.228151,0.217311,0.757526
2558,0.653332,0.387943,0.793178,1.30196,0.716276,0.394747,0.118335,1.410465,0.451186,0.969091,...,0.348772,1.011982,0.191880,1.230596,0.247684,0.946895,0.212751,0.228151,0.217311,0.757526


In [458]:
# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

aux_gamma

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.644010,0.219890,0.259748,1.337019,0.369748,0.454080,0.170702,1.141579,0.417968,0.792131,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
1,0.644010,0.219890,0.259748,1.337019,0.369748,0.454080,0.170702,1.141579,0.417968,0.792131,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
2,0.644010,0.219890,0.259748,1.337019,0.369748,0.454080,0.170702,1.141579,0.417968,0.792131,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
3,0.644010,0.219890,0.259748,1.337019,0.369748,0.454080,0.170702,1.141579,0.417968,0.792131,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
4,0.644010,0.219890,0.259748,1.337019,0.369748,0.454080,0.170702,1.141579,0.417968,0.792131,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2.386378,0.639327,2.774588,5.499775,2.600099,0.792489,0.622851,5.758115,1.269427,4.648178,...,0.567732,3.357565,0.514147,5.270973,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569
2556,2.386378,0.639327,2.774588,5.499775,2.600099,0.792489,0.622851,5.758115,1.269427,4.648178,...,0.567732,3.357565,0.514147,5.270973,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569
2557,2.386378,0.639327,2.774588,5.499775,2.600099,0.792489,0.622851,5.758115,1.269427,4.648178,...,0.567732,3.357565,0.514147,5.270973,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569
2558,2.386378,0.639327,2.774588,5.499775,2.600099,0.792489,0.622851,5.758115,1.269427,4.648178,...,0.567732,3.357565,0.514147,5.270973,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569


In [459]:
df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

aux_labels

,valence,arousal,dominance,liking
0,6.79,5.97,8.08,8.09
1,6.79,5.97,8.08,8.09
2,6.79,5.97,8.08,8.09
3,6.79,5.97,8.08,8.09
4,6.79,5.97,8.08,8.09
...,...,...,...,...
2555,3.99,8.06,2.09,2.03
2556,3.99,8.06,2.09,2.03
2557,3.99,8.06,2.09,2.03
2558,3.99,8.06,2.09,2.03


In [460]:
data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

# Todas las bandas una al lado de otra

In [461]:
frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
aux_all_bands

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
1,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
2,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
3,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
4,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.234428,0.496717,0.138547,0.997258,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,0.567732,3.357565,0.514147,5.270973,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569
2556,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,0.567732,3.357565,0.514147,5.270973,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569
2557,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,0.567732,3.357565,0.514147,5.270973,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569
2558,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,0.567732,3.357565,0.514147,5.270973,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569


In [462]:
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)
all_bands_df

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,F4,F8,AF4,Fp2,Fz,Cz,valence,arousal,dominance,liking
0,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960,6.79,5.97,8.08,8.09
1,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960,6.79,5.97,8.08,8.09
2,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960,6.79,5.97,8.08,8.09
3,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960,6.79,5.97,8.08,8.09
4,0.322804,0.174132,0.138156,1.060014,0.171570,0.195928,0.093503,0.695433,0.259882,0.574426,...,0.330760,0.460760,0.289392,0.301249,0.296073,0.410960,6.79,5.97,8.08,8.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569,3.99,8.06,2.09,2.03
2556,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569,3.99,8.06,2.09,2.03
2557,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569,3.99,8.06,2.09,2.03
2558,0.149376,0.074179,0.258360,0.234226,0.163226,0.153782,0.082766,0.213215,0.091083,0.139919,...,1.135358,4.029013,1.195415,0.678252,0.888841,2.526569,3.99,8.06,2.09,2.03


In [463]:
data = all_bands_df.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, yTrain)

predict = clf.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC(kernel='linear')
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 7

knn = KNeighborsClassifier(n_neighbors)
knn.fit(xTrain, np.ravel(yTrain))
knn_predict = knn.predict(xTest)

rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")

<ipython-input-463-18ea7da0e8e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')
<ipython-input-463-18ea7da0e8e8>:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100

In [464]:
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 3, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 3, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 3, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment3.csv", mode="a", header=False, index=False)